# Project: Stochastic and Spatial Models

## 0. Init

Most code will reference algorithms and practices as described in "Modeling infectious diseases in humans and animals" (MID) by M. Keeling and P. Rohani.

In [ ]:
# WARNING: Comment if code doesn't run
# Use jit to compile and optimize Python code
from numba import jit

# Arrays and analysis
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit

# Plotting and config
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [6, 3]
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['figure.constrained_layout.use'] = True

# Misc imports
from slugify import slugify
from functools import partial
import os

# Important directories
FIG_DIR = 'fig/'
DATA_DIR = 'data/'
DUMP_DIR = 'dump/'

def save_fig(title):
    """Save figure under normalized name."""
    plt.savefig(f'{FIG_DIR}/{slugify(title)}.png', bbox_inches='tight')

def props_fig(props_dict):
    """Update current Axes with the given dictionary of properties."""
    plt.gca().update(props_dict)

def create_dirs(path):
    """Create directory, do nothing if it exists."""
    os.makedirs(path, exist_ok=True)

create_dirs(FIG_DIR)
create_dirs(DATA_DIR)
create_dirs(DUMP_DIR)

## 1. Gillespie’s Direct Algorithm and Stochastic Hallmarks

### 1.1 Implementation of GA and comparison

In [ ]:
# WARNING: Comment if code doesn't run
# Use jit to compile and optimize Python code
@jit(nopython=True)
def SIR_GDA(y0, params, t_max, noise_var=None):
    """Implementation of Gillespie's Direct Algorithm (GDA)
    for the standard SIR model. Written after Box 6.3 in MID (p 201)."""
    # np.random.seed()
    X, Y, Z = y0
    beta, gamma, mu = params
    
    # 1.
    # Events.
    ps = np.array([
        (1, 0, 0),    # birth
        (-1, 1, 0),   # transmission
        (0, -1, 1),   # recovery
        (-1, 0, 0),   # death_X
        (0, -1, 0),   # death_Y
        (0, 0, -1),   # death_Z
    ])

    # Bookkeeping variables.
    t = 0
    ts = []
    ys = []
    while t < t_max:    
        # 2.
        N = np.sum(np.array([X, Y, Z]))

        # Scale the transmission rate by random noise if provided.
        # Appears to be incorrect.
        R_transmission = beta*X*Y/N
        if noise_var is not None:
            xi = np.random.normal(0, noise_var)
            R_transmission = R_transmission + X*Y/N*xi

            # Cap transmission rate to 0.
            if R_transmission < 0:
                R_transmission = 0

        # Rates in the same order as their corresponding events.
        Rs = np.array([mu*N, R_transmission, gamma*Y, mu*X, mu*Y, mu*Z])
        
        # 3.
        R_total = np.sum(Rs)
        
        # 4.
        rand_1 = np.random.rand()
        dt = -1 / R_total * np.log(rand_1)
        
        # 5.
        rand_2 = np.random.rand()
        P = rand_2 * R_total
        
        # 6.
        R_cum = np.cumsum(Rs)
        
        # Find index of event p.
        p_idx = np.searchsorted(R_cum, P)
        
        # Add event result to current state.
        p = ps[p_idx]
        X, Y, Z = np.array([X, Y, Z]) + p
        
        # 7.
        t += dt

        # Bookkeep results.
        ts.append(t); ys.append([X, Y, Z])

    return np.array(ts), np.array(ys)

In [ ]:
def SIR_D(t, y0, params):
    """Deterministic, density-dependent implementation of the SIR model
    with demography as described in MID 2.1.2."""
    X, Y, Z = y0
    beta, gamma, mu = params

    N = X + Y + Z
    dx = mu*N - beta*X*Y/N - mu*X
    dy = beta*X*Y/N - gamma*Y - mu*Y
    dz = gamma*Y - mu*Z
    
    return dx, dy, dz

In [ ]:
title="Comparison of GDA with deterministic SIR model"
t_max = 300
params = (0.3, 0.1, 1/80)
y0 = (990, 10, 0)
num_runs = 100

fig, ax = plt.subplots()

for i in range(num_runs):
    ts, ys = SIR_GDA(y0=y0, params=params, t_max=t_max)
    
    # Plot the results for each trajectory
    plt.plot(ts, ys[:, 0], color='r', alpha=0.1)   
    plt.plot(ts, ys[:, 1], color='g', alpha=0.1)    
    plt.plot(ts, ys[:, 2], color='b', alpha=0.1)

ts_det = np.linspace(0, t_max, 200)
sol = solve_ivp(SIR_D, [0, t_max], y0, args=(params,), dense_output=True)
ys = sol.sol(ts_det).T

plt.plot(ts_det, ys[:, 0], color='#b30000', linewidth=3, label='Susceptible')
plt.plot(ts_det, ys[:, 1], color='#006600', linewidth=3, label='Infected')
plt.plot(ts_det, ys[:, 2], color='#001f5b', linewidth=3, label='Recovered')

plt.legend(loc="upper right")
props_fig({"xlabel": "t (days)", "ylabel": "population", "title": title})
save_fig(title)
plt.show()

### 1.2 Simulation variability and negative covariance

In [ ]:
def errorbar_plot(x, ys, fig_props):
    """Create an errorbar plot where observations over x 
    are collected in the rows of ys."""
    fig, ax = plt.subplots()
    
    ax.errorbar(x, np.mean(ys, axis=1), np.std(ys, axis=1), marker='.', color='black', capsize=4, linestyle='--')
    # Always use scientific notation.
    ax.ticklabel_format(style='scientific', axis='y', scilimits=(0, 0))
    props_fig(fig_props)
    save_fig(fig_props["title"])
    
    plt.show()

In [ ]:
t_max = 100
y0 = (9990, 10, 0)
num_runs = 20

gamma = 0.1
betas = np.linspace(0.3, 2, 13)

all_covs = []
all_variances = []
for beta in betas:
    params = (beta, gamma, 1/80)

    covs = []
    variances = []
    for _ in range(num_runs):
        ts, ys = SIR_GDA(y0=y0, params=params, t_max=t_max)
    
        covs.append(np.cov(ys[:, 0], ys[:, 1])[0][1])
        variances.append(np.var(ys[:, 1]))

    all_covs.append(covs)
    all_variances.append(variances)

In [ ]:
errorbar_plot(betas, all_variances, {
    "xlabel": "beta", 
    "ylabel": "variance", 
    "title": "Mean variance of I for varying beta"
})

errorbar_plot(betas, all_covs, {
    "xlabel": "beta", 
    "ylabel": "covariance", 
    "title": "Mean covariance of S and I for varying beta"
})

In [ ]:
t_max = 100
num_runs = 20

gamma = 0.1
beta = 0.3
params = (beta, gamma, 1/80)

Ns = np.linspace(1000, 20000, 21)

all_covs = []
all_variances = []
for N in Ns:
    y0 = (N - 10, 10, 0)
    
    covs = []
    variances = []
    for _ in range(num_runs):
        ts, ys = SIR_GDA(y0=y0, params=params, t_max=t_max)
    
        covs.append(np.cov(ys[:, 0], ys[:, 1])[0][1])
        variances.append(np.var(ys[:, 1]))

    all_covs.append(covs)
    all_variances.append(variances)

In [ ]:
errorbar_plot(Ns, all_variances, {
    "xlabel": "N", 
    "ylabel": "variance", 
    "title": "Mean variance of I for varying N"
})

errorbar_plot(Ns, all_covs, {
    "xlabel": "N", 
    "ylabel": "covariance", 
    "title": "Mean covariance of S and I for varying N"
})